In [ ]:
import numpy as np
import pickle as pkl
import tensorflow as tf

2024-04-13 03:32:42.202307: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 03:32:42.250745: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 03:32:42.250781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 03:32:42.252447: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 03:32:42.260929: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-13 03:32:42.261723: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [ ]:
# convert to prechunked format
MAX_SPEED=0.2
FRAME_RATE=4

def transform(traj):
    traj_len = len(traj['position'])

    # convert to GNM format -- divide by max_speed/frame_rate to get relative positions from true positions
    poses = []
    for (x,y) in traj['position']:
        new_pos = [x / (MAX_SPEED/FRAME_RATE), y / (MAX_SPEED/FRAME_RATE)] # standar
        poses.append(new_pos)
    
    traj['position'] = np.array(poses)
    
    action_horizon = 4

    # compute rot matrix
    yaw = traj["yaw"] 
    # insert empty axis to front of yaw
    yaw = [yaw for _ in range(traj_len)]
    # Create the rotation matrix using numpy functions for cosine and sine
    rot_mat = tf.convert_to_tensor(
        [
            [tf.cos(yaw), -tf.sin(yaw)],
            [tf.sin(yaw), tf.cos(yaw)],
        ]
    )
    rot_mat = tf.transpose(rot_mat, [3, 2, 0, 1])[0]

    # chunk actions and recompute as relative to the start of the chunk
    pos = traj["position"]
    start = tf.broadcast_to(pos[:, None], [traj_len, action_horizon, 2])
    end_indices = tf.range(traj_len)[:, None] + tf.range(1, action_horizon + 1)
    end_indices = tf.minimum(end_indices, traj_len - 1)
    end = tf.gather(pos, end_indices)
    delta = end - start
    actions = tf.matmul(delta[:, :, None], rot_mat[:, None])[:, :, 0] #* scaling_factor

    return actions


In [ ]:
folders = ['locobot_bww_large_loop_0', 'locobot_crash_test_0', 'locobot_crash_test_1', 'locobot_crash_test_2', 'locobot_crash_test_3', 'locobot_svl_loop_0', 'locobot_svl_loop_1']

pos_data = []
pth = '/nfs/nfs1/users/riadoshi/orca/notebooks/files/locobot_loop/'
for folder in folders:
    with open(pth + folder + '/traj_data.pkl', 'rb') as f:
        data = pkl.load(f)
        data = transform(data)
        print(data.shape)
        pos_data.extend(data)
pos_data = np.array(pos_data)

(1892, 4, 2)
(177, 4, 2)
(171, 4, 2)
(154, 4, 2)
(184, 4, 2)
(763, 4, 2)
(6, 4, 2)


In [ ]:
pos_data.shape
# actually now that i think about it, gnm data is already relative. and we're applying this transform on top of that. so we want to divide by max_speed/frame_rate
# so divide by 0.2/4

(3347, 4, 2)

In [ ]:
xs, ys = [], []
for chunk in pos_data:
    # we apply the transform assuming that the data is 
    x,y = chunk[0][0], chunk[0][1]
    xs.append(x)
    ys.append(y)

xs, ys = np.array(xs), np.array(ys)


In [ ]:
xs.mean(), ys.mean()

(0.26631192613091664, 0.22549576019925752)

In [ ]:
xs.std(), ys.std()

(1.309577033271587, 1.3695118288543364)